In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

### Access the website https://db-engines.com/en/ranking, perform web scraping, and generate a DataFrame with the list of ranked databases.

### Using Seaborn, create a bar chart showing the database names and their scores for the current month.

### Create a new DataFrame with only the top 10 databases. Add a column Share showing each database's percentage of relevance relative to the top 10 total scores. Display the name and percentage of the top-ranked database.

### Using Matplotlib, create a chart showing the Share of each database among the top 10.

### Using the full DataFrame, create a new Share column showing the percentage relevance of each database relative to all others.

### Save the full DataFrame as a CSV file named db-ranking.csv.

### Use the Brazilian Central Bank’s API to get the USD exchange rate. Retrieve the selling rate for September 16, 2008, using JSON.